# 감성 분석 테스트 데이터 구축

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

import re
import numpy as np
import pandas as pd

In [107]:
def getUrl(bs, stockcode):
    
    # 맨 마지막 페이지 번호 가져오기
    for link in bs.find_all('a'):
        if '맨뒤' in link.text:
            n = int(link['href'].split('page=')[1].split('&')[0])
    
    urls = []
    for i in range(1,n+1):
        url = 'https://finance.naver.com/item/news_news.naver?code=' + stockcode + '&page='+ str(i)+'&sm=title_entity_id.basic&clusterId='
        urls.append(url)
    if urls:
        print('url들을 불러왔습니다')
        
    return urls




def getLinks(urls, includeUrl):

    internalLinks = []
    
    for url in urls:
        html = urlopen(url)
        bs = BeautifulSoup(html.read(), 'html.parser')

        # 내부 링크
        for link in bs.find_all('a'):
            if link is not None:
                if(link.attrs['href'].startswith(includeUrl)):
                    if link not in internalLinks:
                        url = 'https://finance.naver.com/'+link.attrs['href']
                        internalLinks.append(url)
                        
    if internalLinks:
        print(len(internalLinks),'개 링크 수집 완료')
    
        return internalLinks
      

def crawl_news(stockcode, internalLinks):
    
    news = []
    
    for i, link in enumerate(internalLinks):
        try:
            print(link)
            html = urlopen(link)
            bs = BeautifulSoup(html.read(), 'html.parser')

            # 제목
            title = bs.tbody.find('th').text

            # 신문사
            newspaper = bs.tbody.findAll('th')[1].span.text.split('  ')[0]

            # 날짜 시간
            datetime = bs.tbody.findAll('th')[1].span.text.split('  ')[1]

            # 기사 내용
            content = bs.find('div', {'id': 'news_read'})
            content.find('div',{'class':"link_news"}).extract()
            content = re.sub("\n|\t","",content.text)

            news.append([title, newspaper, datetime, content, stockcode, link])

            if i % 10 ==0:
                print(i,'번째 기사 수집중----------------------')

        except AttributeError:
            continue

    
    return news
        
    
    

In [108]:
from pymongo import MongoClient
import numpy as np
import pandas as pd

def crawl_news_to_db(stockcode = '005930'):
    
    url = 'https://finance.naver.com/item/news_news.naver?code=005930&page=&sm=title_entity_id.basic&clusterId='
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    includeUrl = '/item/news_read.naver?article_id'

    urls = getUrl(bs, stockcode= '005930')
    internalLinks = getLinks(urls, includeUrl)
    news = crawl_news(stockcode, internalLinks)
    
    #localhost에 연결
    #client = MongoClient('localhost', 27017)

    #db = client.donghakgaemi

    df = pd.DataFrame(news, columns = ['title', 'newspaper', 'datetime', 'content', 'stockcode', 'link'])
    
    df.to_csv(stockcode+'.csv',index=False)
    docs = df.to_dict("records")

    #for doc in docs:
    #    db.news.insert_one(doc)


In [109]:
# 삼성 전자 
crawl_news_to_db(stockcode = '005930')

url들을 불러왔습니다


KeyboardInterrupt: 

In [93]:
if __name__ == '__main__':
    codes = ["035420", "005930", "090430", "005380", "051910", "035720", "005490", "302440",
            "036570", "034020", "011200", "020560", "009830", "004170", "041510",
            "000100", "293490", "000660", "068270", "012330"]

    for code in codes:
        crawl_news_to_db(code)

url들을 불러왔습니다


KeyboardInterrupt: 

* 목록에서 기사 제목, 신문사, 날짜시간 가져오는 코드 추가

In [ ]:
def getUrl(bs, stockcode):
    
    # 맨 마지막 페이지 번호 가져오기
    for link in bs.find_all('a'):
        if '맨뒤' in link.text:
            n = int(link['href'].split('page=')[1].split('&')[0])
    
    urls = []
    n=1
    for i in range(1,n+1):
        url = 'https://finance.naver.com/item/news_news.naver?code=' + stockcode + '&page='+ str(i)+'&sm=title_entity_id.basic&clusterId='
        urls.append(url)
    if urls:
        print('url들을 불러왔습니다')
        
    return urls

def getNewsInfo(urls, includeUrl):

    
    # [제목], 신문사, 날짜 시간, 내부 링크 저장
    #titles = []
    #newspapers = []
    #datetimes = []
    internalLinks = []
    
    for url in urls:
        html = urlopen(url)
        bs = BeautifulSoup(html.read(), 'html.parser')
        
        # title
        for title in bs.find_all('td', class_='title'):
            titles.append(title.text)
        
        # 신문사 
        for newspaper in bs.find_all('td', class_='info'):
            newspapers.append(newspaper.text)
            
        # 날짜 + 시간
        for dt in bs.find_all('td', class_='date'):
            datetimes.append(dt.text)
        
        # 내부 링크
        for link in bs.find_all('a'):
            if link is not None:
                if(link.attrs['href'].startswith(includeUrl)):
                    if link not in internalLinks:
                        url = 'https://finance.naver.com/'+link.attrs['href']
                        internalLinks.append(url)
    
    
    return newspapers, datetimes, internalLinks

def crawl_news(internalLinks):
    
    titles = []
    contents = []
    newspapers = []
    datetimes = []
    

    for i, link in enumerate(internalLinks):
        print(link)
        html = urlopen(link)
        bs = BeautifulSoup(html.read(), 'html.parser')

        # 기사 내용
        content = bs.find('div', {'id': 'news_read'})
        content.find('div',{'class':"link_news"}).extract()
        content = re.sub("\n|\t","",content.text)
        contents.append(content)


        print(i,'번째 기사 수집중----------------------',title,newspaper,datetime,content)

    
    return contents

In [2]:
url = 'https://finance.naver.com/item/news_news.naver?code=005930&page=1300&sm=title_entity_id.basic&clusterId='
html = urlopen(url)
bs = BeautifulSoup(html.read(), 'html.parser')

In [3]:
bs

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211029233853/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script src="https

In [4]:
import pandas as pd

In [14]:
df = pd.read_csv('005930.csv')

In [15]:
df.head()

,title,newspaper,datetime,content,stockcode,link
0,"""이거 진짜 괜찮았는데""…LG 롤러블폰, 삼성이 만드나 [배성수의 다다IT선]",한국경제,2021.11.13 15:36,삼성 최신 특허 보니슬라이더블 디스플레이 컨셉 영상/사진=삼성디스플레이삼성전자의 폴...,5930,https://finance.naver.com//item/news_read.nave...
1,“종이 비행기나 접어라” ‘조롱’ 받던 삼성 ‘놀라운 반전’,헤럴드경제,2021.11.13 14:41,"2019년 4월, 월스트리저널은 '갤럭시폴드 5G'의 내구성을 비판하는 영상을 게시...",5930,https://finance.naver.com//item/news_read.nave...
2,삼성전자가 美에 제출한 영업기밀은?[배진솔의 전자사전],이데일리,2021.11.13 12:01,"8일 美 상무부에 반도체·자동차 업체 자료 제출 마감SK하이닉스, 메모리반도체 ""생...",5930,https://finance.naver.com//item/news_read.nave...
3,삼성전자 미국 투자발표 초읽기…미중 정상회담 변수 되나,매일경제,2021.11.13 10:45,"백악관 ""미중 화상정상회담 15일 저녁 개최"" 공식확인한국시간으로 16일 오전......",5930,https://finance.naver.com//item/news_read.nave...
4,"""삼성, 내년 1월 갤S21 FE, 2월 갤S22 공개"" 전망",헤럴드경제,2021.11.13 09:24,[헤럴드경제] 삼성전자가 당초 올해 하반기 출시할 것으로 점쳐졌던 보급형 모델 갤...,5930,https://finance.naver.com//item/news_read.nave...


In [10]:
df.columns = ['title', 'newspaper', 'datetime', 'content', 'stockcode', 'link']

In [11]:
df.head()

,title,newspaper,datetime,content,stockcode,link
0,"""이거 진짜 괜찮았는데""…LG 롤러블폰, 삼성이 만드나 [배성수의 다다IT선]",한국경제,2021.11.13 15:36,삼성 최신 특허 보니슬라이더블 디스플레이 컨셉 영상/사진=삼성디스플레이삼성전자의 폴...,5930,https://finance.naver.com//item/news_read.nave...
1,“종이 비행기나 접어라” ‘조롱’ 받던 삼성 ‘놀라운 반전’,헤럴드경제,2021.11.13 14:41,"2019년 4월, 월스트리저널은 '갤럭시폴드 5G'의 내구성을 비판하는 영상을 게시...",5930,https://finance.naver.com//item/news_read.nave...
2,삼성전자가 美에 제출한 영업기밀은?[배진솔의 전자사전],이데일리,2021.11.13 12:01,"8일 美 상무부에 반도체·자동차 업체 자료 제출 마감SK하이닉스, 메모리반도체 ""생...",5930,https://finance.naver.com//item/news_read.nave...
3,삼성전자 미국 투자발표 초읽기…미중 정상회담 변수 되나,매일경제,2021.11.13 10:45,"백악관 ""미중 화상정상회담 15일 저녁 개최"" 공식확인한국시간으로 16일 오전......",5930,https://finance.naver.com//item/news_read.nave...
4,"""삼성, 내년 1월 갤S21 FE, 2월 갤S22 공개"" 전망",헤럴드경제,2021.11.13 09:24,[헤럴드경제] 삼성전자가 당초 올해 하반기 출시할 것으로 점쳐졌던 보급형 모델 갤...,5930,https://finance.naver.com//item/news_read.nave...


In [12]:
df.shape

(25223, 6)

In [13]:
df.to_csv('005930.csv',index=False)